In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

In [2]:
# Load dataset
file_path = 'dataset.csv'
df = pd.read_csv(file_path, encoding='latin-1', sep=';')

In [18]:
# Extract columns
kuno_texts = df['Kuno'].astype(str).values
ngoko_texts = df['Ngoko'].astype(str).values

In [19]:
# Gabungkan semua teks untuk tokenisasi
all_texts = np.concatenate([kuno_texts, ngoko_texts])

In [20]:

# Gunakan satu tokenizer untuk semua teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

In [21]:
# Ukuran vocabulary
vocab_size = len(tokenizer.word_index) + 1

In [22]:
# Konversi teks ke sequence
kuno_sequences = tokenizer.texts_to_sequences(kuno_texts)
ngoko_sequences = tokenizer.texts_to_sequences(ngoko_texts)

In [24]:
# Padding
kuno_padded = pad_sequences(kuno_sequences, maxlen=max_sequence_length, padding='post')
ngoko_padded = pad_sequences(ngoko_sequences, maxlen=max_sequence_length, padding='post')

In [25]:

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    kuno_padded, ngoko_padded, test_size=0.2, random_state=42
)

In [26]:
# Konfigurasi model
embedding_dim = 128
latent_dim = 256

In [27]:
# Encoder
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

In [28]:
# Decoder
decoder_inputs = Input(shape=(max_sequence_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True)(
    decoder_embedding, initial_state=[state_h, state_c]
)

In [29]:
# Output layer
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

In [30]:
# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
# Persiapan label
y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)

In [ ]:
# Training
model.fit(
    [X_train, X_train], y_train, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)


Epoch 1/50
4/4 [==============================] - 27s 2s/step - loss: 6.5007 - accuracy: 0.0377 - val_loss: 6.4953 - val_accuracy: 0.1111
Epoch 2/50
4/4 [==============================] - 0s 103ms/step - loss: 6.4818 - accuracy: 0.1038 - val_loss: 6.4843 - val_accuracy: 0.1019
Epoch 3/50
4/4 [==============================] - 0s 101ms/step - loss: 6.4519 - accuracy: 0.0943 - val_loss: 6.4568 - val_accuracy: 0.1019
Epoch 4/50
4/4 [==============================] - 0s 105ms/step - loss: 6.3684 - accuracy: 0.0849 - val_loss: 6.3688 - val_accuracy: 0.1019
Epoch 5/50
4/4 [==============================] - 0s 104ms/step - loss: 6.0854 - accuracy: 0.0825 - val_loss: 6.1422 - val_accuracy: 0.1019
Epoch 6/50
4/4 [==============================] - 0s 104ms/step - loss: 5.6098 - accuracy: 0.0825 - val_loss: 6.1521 - val_accuracy: 0.1019
Epoch 7/50
4/4 [==============================] - 0s 104ms/step - loss: 5.2697 - accuracy: 0.0849 - val_loss: 6.3410 - val_accuracy: 0.1111
Epoch 8/50
4/4 [======

In [ ]:
# Fungsi untuk menterjemahkan
def translate_text(input_text):
    # Konversi input ke sequence
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')
    
    # Prediksi
    predicted_output = input_padded.copy()
    
    for i in range(max_sequence_length):
        predictions = model.predict([input_padded, predicted_output])
        predicted_idx = np.argmax(predictions[0, i, :])
        predicted_output[0, i] = predicted_idx
    
    # Konversi kembali ke teks
    output_words = [tokenizer.index_word.get(idx, '') for idx in predicted_output[0] if idx != 0]
    return ' '.join(output_words)



In [ ]:
# Evaluasi model
test_accuracy = model.evaluate([X_test, X_test], y_test)[1]
print(f'Akurasi model: {test_accuracy:.2f}')



2/2 [==============================] - 0s 51ms/step - loss: 7.3609 - accuracy: 0.2400
Akurasi model: 0.24


In [37]:
# Interaksi pengguna
while True:
    user_input = input("Masukkan teks Kuno untuk diterjemahkan ke Ngoko (atau ketik 'exit' untuk keluar): ")
    if user_input.lower() == 'exit':
        break
    
    translation = translate_text(user_input)
    print(f"Hasil terjemahan: {translation}")

1/1 [==============================] - 0s 62ms/step
Hasil terjemahan: dharma tama anane bener anane anane anane
